A famous early application of low-number statistics was an analysis of Prussian cavalryman horse-kick deaths by Bortkiewicz in 1898.

He studied the distribution of 122 men kicked to death by horses among 10 Prussian army corps within 20 years (so a total number of corpes of 200). He recorded the number of corps with a given number of deaths:

Number of deaths	Number of groups
        0	                109
        1	                65
        2	                22
        3	                3
        4	                1
Plot the resulting probability distribution. Careful with the normalization
How does it look like? Again try to fit it by hand.